### Import relevant packages/libraries

In [1]:
import cv2
import numpy as np
# the following packages are specific to the Raspberry Pi (Rpi)
# import RPi.GPIO as GPIO
# from time import sleep

### Functions

In [2]:
def distt(x1,y1,x2,y2): # general function for finding Euclidean distance between two points
    return np.sqrt(np.square(np.absolute(x2-x1)) + np.square(np.absolute(y2-y1)))

In [3]:
def distance_to_camera(knownWidth, focalLength, perWidth):
    return (knownWidth * focalLength) / perWidth # compute and return the distance of the object from the camera

In [4]:
# Specific to Rpi, set initial state of LEDs 
# GPIO.setwarnings(False)
# GPIO.setmode(GPIO.BOARD)
# GPIO.setup(8, GPIO.OUT, initial=GPIO.LOW)
# GPIO.setup(10, GPIO.OUT, initial=GPIO.LOW)

In [5]:
classNames= [] # variable used for names of class labels
classFile = 'coco.names' # text file containing labels
with open(classFile,'rt') as f:  # open file and parse class labels
    classNames = f.read().rstrip('\n').split('\n')

configPath = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt' # network configuration
weightsPath = 'frozen_inference_graph.pb' # trained weights

net = cv2.dnn_DetectionModel(weightsPath,configPath) # create detection model usng trained weights and network configuration
# scaling and pixel intensity parameters
net.setInputSize(320,320)
net.setInputScale(1.0/ 127.5)
net.setInputMean((127.5, 127.5, 127.5)) 
net.setInputSwapRB(True)

def getObjects(img,thres,nms,ped_width,cyc_width,focalLength,draw=True,objects=[]): # object detection function
    # nms parameter dictates multiple detections and supresses the boundng boxes with non-maximum confidence level
    dist_centres = 0 # avoid local variable assignment problems
    counter2 = 0 # used to count number of object detected over entire video/webcam duration, initiliase to 0 at every iteration
    counter3 = 0 # used for cumulative value of all confidence levels throughout runtime, initiliase to 0 at every iteration
    classIds, confs, bbox = net.detect(img,confThreshold=thres,nmsThreshold=nms) # retrieve details about objects detected
    if len(objects) == 0: # if user did not define any specific objects to detect 
        objects = classNames # detect all objects within coco.names file
    objectInfo =[] # create variable for object information (confidence level...)
    if len(classIds) != 0: # if object detected
        cyc_centrex=0 # avoid local variable assignment problems
        cyc_centrey=0 # avoid local variable assignment problems
        for i in range(classIds.shape[0]): # iterate through class IDs
            if classIds[i] == 2: # if object detected is bicycle
                counter = i # save specific index for later use 
                box_temp = bbox[i] # save bounding box coordinates of bicycle
                cyc_centrex = box_temp[0]+(box_temp[2]/2) # centre bicycle x coord
                cyc_centrey = box_temp[1]+(box_temp[3]/2) # centre bicycle y coord
                break
        dist_centres = 0 # avoid local variable assignment problems
        ped_centrex = 0 # avoid local variable assignment problems
        ped_centrey = 0 # avoid local variable assignment problems
        for classId, confidence,box in zip(classIds.flatten(),confs.flatten(),bbox):  # iterates through multiple objects
            className = classNames[classId - 1] # retrieve class label name
            counter2 += 1 # increment by 1 for each object detected
            counter3 += confidence # add current confidence value to running total
            if className in objects: # if object detected is within user-specified list
                objectInfo.append([box, className]) # if object within class list, append name and set of bounding boxes
                dist = distance_to_camera(ped_width, focalLength, box[2]) # distance calculation for pedestrian
# RASPBERRY PI LED alert
#                 if dist < 200 and classIds[i] == 2: # covers any situation where cyclist in frame (multiple objects...)
#                     GPIO.output(8, GPIO.HIGH) # light LED 1
#                     GPIO.output(10, GPIO.HIGH) # light LED 2
#                     sleep(0.5) # ensures LED turns off 0.5 seconds after cyclist(s) have passed, will keep resetting if cyclist still in frame
#                     GPIO.output(8, GPIO.LOW) # turn off LED 1
#                     GPIO.output(10, GPIO.LOW) # turn off LED 2
                    
#                 else: # covers situation where only pedestrians have been detected, located within if statement where object has been detected already                    
#                     GPIO.output(8, GPIO.HIGH) # light LED 1                  
#                     sleep(0.5) # ensures LED turns off 0.5 seconds after pedestrian(s) have passed
#                     GPIO.output(8, GPIO.LOW) # turn off LED 1
                
                ped_centrex = box[0]+(box[2]/2) # centre pedestrian x coord
                ped_centrey = box[1]+(box[3]/2) # centre pedestrian y coord
                dist_centres = distt(ped_centrex,ped_centrey,cyc_centrex,cyc_centrey) # calculate distance from bicycle to pedestrian
                if dist_centres < 150: # if person close enough to bicycle change label to cyclist
                    if dist_centres == 0: # avoids bicycle label itself becoming cyclist
                        break
                    else:
                        className = 'cyclist' # change class label to 'cyclist'
                        dist = distance_to_camera(cyc_width, focalLength, box[2]) # distance calculation for cyclist, replaces pedestrian distance
                if (draw): # initially True
                    cv2.rectangle(img,box,color=(0,255,0),thickness=2) # display bounding box
                    cv2.putText(img,className.upper(),(box[0]+10,box[1]+30),
                    cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,0),2) # display object class name
                    if box[3] > 200: # if box large enough, avoids overcrowding the screen
                        cv2.putText(img,str(round(confidence*100,2)),(box[0]+10,box[1]+60),
                        cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,0),2) # display confidence level
                        cv2.putText(img,"%.2f in" % dist,(box[0]+10,box[1]+90),
                        cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,0),2) # display distance 
                    
    return img,objectInfo,counter2,counter3 # return useful figures

'bbox' is simply just an array of pixel values (top left corner x, top left corner y, width, height) of the bounding boxes of ALL objects within image, '.flatten' collapses matrix into one dimension, zip joins each 1D array of classIds, confs and bbox values together in single tuple, simply just a way to iterate through and extract the relevant information

### Image, Video, Webcam 

In [6]:
# Focal length calibration
# KNOWN_DISTANCE = 9 # inches, used for initial calibration, corresponds to fork, spoon, phone, mouse... images
# focalLength = (p[2] * KNOWN_DISTANCE) / KNOWN_WIDTH # triangle similarity formula rearranged, average of many of these values used

In [7]:
# Code to run algorithm on STILL IMAGE
KNOWN_PEDWIDTH = 20 # human average, adjusted after evaluation and testing
KNOWN_CYCWIDTH = 68 # average width of bicycle, adjusted after evaluation and testing
focalLength = 827 # calculated in calibration

cv2.namedWindow("output", cv2.WINDOW_NORMAL) # Create window with freedom of dimensions
image1 = cv2.imread("Cyclist1.png") # read in image
image = cv2.resize(image1, (1200, 650)) # Resize image to fit on screen
# call 'getObjects' function'on image, threshold value of 0.45, nms value of 0.2, specifiy 'person' and 'bicycle' as objects to be detected
result,objectInfo,c2,c3 = getObjects(image,0.45,0.2,KNOWN_PEDWIDTH,KNOWN_CYCWIDTH,focalLength,objects=['person','bicycle']) 
cv2.imshow("Output", image) # output image to screen
cv2.imwrite("Cyclist11.jpg", image) # save resulting image
cv2.waitKey(0) # wait for key to be pressed to close window
cv2.destroyAllWindows() # appropriately close window

In [8]:
# Code to run algorithm on VIDEO

counter1 = 0 # variable used to measure frames per second of algorithm, benchmark value
counter2 = 0 # used to count number of object detected over entire video/webcam duration
counter3 = 0 # used for cumulative value of all confidence levels throughout runtime
KNOWN_PEDWIDTH = 20 
KNOWN_CYCWIDTH = 68  
focalLength = 827

cap = cv2.VideoCapture(0) # create 'VideoCapture' object, 'Cycling_vid_Trim_Trim.avi'
# set input paramters (height and width)
cap.set(3, 640)
cap.set(4, 480)

# We need to set resolutions for VideoWriter function convert them from float to integer. 
frame_width = int(cap.get(3)) 
frame_height = int(cap.get(4)) 
size = (frame_width, frame_height) 
# Below 'VideoWriter' object will create a frame of above, the output is stored as video file
vid = cv2.VideoWriter('CycPed.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size) # save video of detected objects

try:
    while True: # constantly reads individual frames 
        success, img = cap.read() # first variable is 0 or 1 depending on whether video successfully read, second is video itself i.e. image frames
        if success == 1: # error handling   
            # call 'getObjects' function on iterated frames
            result,objectInfo,c1,c2 = getObjects(img,0.45,0.2,KNOWN_PEDWIDTH,KNOWN_CYCWIDTH,focalLength,objects=[]) # 
            vid.write(img) # write video into file one frame at time 
            counter1 += 1 # number of frames
            counter2 += c1 # number of objects detected within single image
            counter3 += c2 # cumulative confidence levels of all objects
            meanclass_accuracy = counter3/counter2 # mean classifaction accuracy calculation
            cv2.imshow("Output", img) # output images
            if cv2.waitKey(1) & 0xFF == ord('q'): # run through until user presses 'q'
                break
        else: # break while loop if image not read successfully
            break

    cap.release() # release 'VideoCapture' object 
    vid.release() # release 'VideoWriter' object 
    cv2.destroyAllWindows() 
    
except Exception as e: # error handling
    print(str(e))